## Importataion

In [16]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

## 1.modeele GradientBoost

In [65]:
data=pd.read_csv('train_set.csv')
data=data.drop(['moyeneGeneral','Note1', 'Note2', 'Note3','Note4', 'Note5', 'Note6', 'Note7', 'Note8','Nom','ID'],axis=1)
data.head()

,Unnamed: 0,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance
0,27634,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,Ajourné,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998
1,36311,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,Ajourné,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991
2,21059,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,Ajourné,Inchiri,AKJOUJT,8 مايو 1995
3,24532,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,Ajourné,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999
4,17861,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,Ajourné,Dakhlet NDB,NOUADHIBOU,10 mai 2001


## preprocessing

In [66]:
### creer une colonne pour connaitre l'age
data['age'] = data['DateNaissance'].apply(  
    lambda x: int(x.split("-")[2] if '-' in x else int(x.split(" ")[2]) )
    ).apply(lambda x : 2021 - x if x > 1900  else ( 121 - x if x > 21 else 21 - x))

In [67]:
data=data.drop('Unnamed: 0',axis=1)
# Transformer les observations
data['Decision'] = data['Decision'].replace(['Ajourné', 'Sessionnaire', 'Abscent', 'Examen annulé à cause du Téléphone', 'Examen annulé à cause du comportement'], 'Ajourné')
### encoder la variable target 
data['Decision']=pd.get_dummies(data['Decision'],drop_first=True)
data.head()

,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance,age
0,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,1,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998,23
1,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,1,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991,30
2,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,1,Inchiri,AKJOUJT,8 مايو 1995,26
3,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,1,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999,22
4,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,1,Dakhlet NDB,NOUADHIBOU,10 mai 2001,20


In [68]:
# Séparer les variables indépendantes et la variable dépendante
X = data.drop('Decision',axis=1)
y = data['Decision']

In [70]:
# Définir les colonnes catégorielles et numériques
cat_cols = ['Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa']
num_cols = ['age']

Commentaire : ces variables definie sont les variables consideree commme des features (variables expliquatives)

### Implementation du modele

In [72]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd



# Séparer les variables indépendantes et la variable dépendante
X = data.drop('Decision',axis=1)
y = data['Decision']

# Diviser les données en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Définir les colonnes catégorielles et numériques
cat_cols = ['Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa']
num_cols = ['age']

# Transformer les données en objets DataFrame
X_train = pd.DataFrame(X_train, columns=cat_cols+num_cols)
X_test = pd.DataFrame(X_test, columns=cat_cols+num_cols)

# Créer le préprocesseur des données
### encodage des variables categoreils et nornalisation des variables numeriques 
preprocessor = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(handle_unknown='ignore'), cat_cols)
)

# Créer le pipeline complet avec le préprocesseur et le modèle
model = make_pipeline(
    preprocessor,
    GradientBoostingClassifier()
)

# Entraîner le modèle sur les données d'apprentissage
model.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)


In [73]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculer les prédictions sur les données de test
y_pred = model.predict(X_test)

# Calculer les différentes métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

# Afficher les résultats
print("Accuracy : ", accuracy)
print("Precision : ", precision)
print("Recall : ", recall)
print("F1-score : ", f1)
print("AUC-ROC : ", auc_roc)

Accuracy :  0.9213224523958987
Precision :  0.9224283305227656
Recall :  0.9981751824817519
F1-score :  0.9588080631025417
AUC-ROC :  0.7741900928578027


## prediction applique sur la dataset submission_template

j'ai cree dans mon pemiere modele (nayev_bayesienne) un dataframe issue de la concatenation des dataframes train et test puis j'ai selectionne les ID qui sont similaires avec les ID de submission_template puis on l'a enregistre comme csv avec nom 'new_data.csv'

In [43]:
# Charger les données dans un DataFrame pandas
### Importation
new=pd.read_csv('new_data.csv')
new=new.drop('Unnamed: 0',axis=1)
new=new.drop('prediction',axis=1)
new1=new.drop('Nom',axis=1)


# Prétraiter les données
#new_data_processed = preprocessor.transform(new1)
X_test2 = pd.DataFrame(new1, columns=cat_cols+num_cols)

# Faire des prédictions sur les données prétraitées
y_pred2 = model.predict(X_test2)

# Afficher les prédictions
print(y_pred2)

[1 1 1 ... 1 1 1]


In [44]:
g=pd.DataFrame(y_pred2)
g.value_counts()

1    15805
0      125
dtype: int64

In [45]:
##calculer la probabilite d'admission
proba=model.predict_proba(X_test2)

In [46]:
### selectionner la probabilite d'admission
pre=proba[:, 0]
len(pre)

15930

In [47]:
new1['predicted2']=pre

In [48]:
submission=new1[['ID','predicted2']]
submission

,ID,predicted2
0,ID_15,0.037835
1,ID_22,0.107953
2,ID_23,0.107953
3,ID_29,0.041576
4,ID_34,0.041576
...,...,...
15925,ID_13569,0.036362
15926,ID_31670,0.026363
15927,ID_31759,0.042481
15928,ID_5855,0.104046


In [75]:
### enregister comme Csv

In [49]:
submission.to_csv('gradient.csv',index=False)

#### application du modele GradientBoostingClassifier sans utiliser le pipline

In [50]:
# Séparer les variables indépendantes et la variable dépendante
X = data.drop('Decision', axis=1)
y = data['Decision']
# Diviser les données en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Définir les colonnes catégorielles et numériques
cat_cols = ['Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa']
num_cols = ['age']
# Prétraitement des données
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

# Encodage des variables catégorielles dans le jeu de données d'entraînement
X_train = preprocessor.fit_transform(X_train)
# Encodage des variables catégorielles dans le jeu de données de test
X_test = preprocessor.transform(X_test)

# Créer le modèle de Gradient Boosting
gb = GradientBoostingClassifier()

# Entraîner le modèle sur les données d'apprentissage
gb.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = gb.predict(X_test)

In [55]:
new1.head()

,ID,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,age
0,ID_15,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,22
1,ID_22,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18
2,ID_23,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18
3,ID_29,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21
4,ID_34,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,21


In [76]:
### prdiction de submission_template

In [54]:
new=pd.read_csv('new_data.csv')
new=new.drop('Unnamed: 0',axis=1)
new=new.drop('prediction',axis=1)
new1=new.drop('Nom',axis=1)
# Prétraiter les données
new_data_processed = preprocessor.transform(new1)


# Faire des prédictions sur les données prétraitées
y_pred3 = gb.predict(new_data_processed)

# Afficher les prédictions
print(y_pred3)

[1 1 1 ... 1 1 1]


In [57]:
proba=gb.predict_proba(new_data_processed)
pre=proba[:, 0]
len(pre)
new1['predicted2']=pre
submission=new1[['ID','predicted2']]
submission
submission.to_csv('gb.csv',index=False)

In [58]:
submission

,ID,predicted2
0,ID_15,0.035668
1,ID_22,0.102206
2,ID_23,0.102206
3,ID_29,0.039209
4,ID_34,0.039209
...,...,...
15925,ID_13569,0.035317
15926,ID_31670,0.026518
15927,ID_31759,0.042687
15928,ID_5855,0.101269


## 2. Modele du randomforest

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Séparer les variables indépendantes et la variable dépendante
X = data.drop('Decision',axis=1)
y = data['Decision']

# Diviser les données en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Définir les colonnes catégorielles et numériques
cat_cols = ['Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa']
num_cols = ['age']

# Prétraitement des données
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

# Créer le pipeline complet avec le préprocesseur et le modèle
model = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators=100, random_state=42)
)

# Entraîner le modèle sur les données d'apprentissage
model.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle sur les données de test
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.37      0.16      0.22       395
           1       0.93      0.98      0.95      4384

    accuracy                           0.91      4779
   macro avg       0.65      0.57      0.59      4779
weighted avg       0.88      0.91      0.89      4779



In [60]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

### prdiction de submission_template

In [63]:
new=pd.read_csv('new_data.csv')
new=new.drop('Unnamed: 0',axis=1)
new=new.drop('prediction',axis=1)
new_data_processed=new.drop('Nom',axis=1)
# Faire des prédictions sur les données prétraitées
y_pred3 = model.predict(new_data_processed)

# Afficher les prédictions
print(y_pred3)

[1 1 1 ... 1 1 1]


In [77]:
### calculer les probabilites puis fussioner avec les ID du submission_template

In [64]:
proba=model.predict_proba(new_data_processed)
pre=proba[:, 0]
len(pre)
new1['predicted2']=pre
submission=new1[['ID','predicted2']]
submission
submission.to_csv('fr.csv',index=False)